In [1]:
import pandas as pd
dff=pd.read_csv("Combined_clean_final.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Combined_clean_final.csv'

In [39]:
df.head(50)
df.columns

Index(['timestamp', 'client_ip', 'method', 'url', 'path', 'body',
       'attack_type', 'body_len', 'body_entropy', 'special_char_count',
       'label'],
      dtype='object')

In [31]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,label
0,2025-01-01T12:10:00Z,offline,GET,http://test/fi?page=php://filter/convert.base6...,/fi,php://filter/convert.base64-encode/resource=in...,NORMAL,53.0,4.22,9.0,0
1,2025-01-01T12:11:00Z,offline,GET,http://test/fi?page=%2e%2e%2fetc%2fpasswd,/fi,%2e%2e%2fetc%2fpasswd,NORMAL,21.0,4.01,4.0,0
2,2025-01-01T12:12:00Z,offline,GET,http://test/fi?page=..%252f..%252fetc%252fpasswd,/fi,..%252f..%252fetc%252fpasswd,NORMAL,28.0,4.60,7.0,0
3,2025-01-01T12:13:00Z,offline,GET,http://test/fi?page=..%2f..%2f..%2fetc%2fpasswd,/fi,..%2f..%2f..%2fetc%2fpasswd,NORMAL,27.0,5.57,10.0,0
4,2025-01-01T12:14:00Z,offline,GET,http://test/fi?page=../../../../etc/passwd,/fi,../../../../etc/passwd,NORMAL,22.0,4.39,13.0,0


In [32]:
df.to_csv("final_attack_type.csv", index=False)


In [33]:
import pandas as pd
import re
from urllib.parse import unquote_plus, urlparse, parse_qs

# =========================
# NORMALIZATION
# =========================
def normalize(text):
    if not isinstance(text, str):
        return ""
    for _ in range(3):
        text = unquote_plus(text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# =========================
# SIGNATURES
# =========================
SQLI_SIGNS = [" or 1=1", "union select", "sleep(", "benchmark(", "--"]
CMD_SIGNS  = [";", "&&", "||", "|", "`", "$("]
PATH_SIGNS = ["../", "..\\", "php://", "file://", "data://", "/etc/passwd"]
XSS_SIGNS  = ["<script", "</script>", "javascript:", "onerror=", "onload="]

ALLOWED_UPLOAD_EXT = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt", "text"]
BLOCKED_UPLOAD_EXT = [".php", ".py", ".exe", ".jsp", ".sh", "js",
                      ".bash", ".c", ".asm", ".java", ".class"]

# =========================
# FILE UPLOAD CHECK (CSV SAFE)
# =========================
def check_file_upload_csv(headers, body):
    if "multipart/form-data" not in headers.lower():
        return None

    matches = re.findall(r'filename="([^"]+)"', body, re.IGNORECASE)

    for fname in matches:
        fname = fname.lower()

        for ext in BLOCKED_UPLOAD_EXT:
            if fname.endswith(ext):
                return "FILE_UPLOAD_ABUSE"

        if fname.count(".") >= 2:
            return "FILE_UPLOAD_ABUSE"

        for ext in ALLOWED_UPLOAD_EXT:
            if fname.endswith(ext):
                return "NORMAL"

        return "FILE_UPLOAD_ABUSE"

    return "NORMAL"

# =========================
# OTHER ATTACKS (CSV SAFE)
# =========================
def check_other_attacks_csv(url, body):
    attacks = []

    parsed = urlparse(url)
    query_vals = []

    for v in parse_qs(parsed.query).values():
        query_vals.extend(v)

    inputs = query_vals + [body]
    norm_inputs = [normalize(i) for i in inputs]

    if any(any(s in d for s in SQLI_SIGNS) for d in norm_inputs):
        attacks.append("SQL_INJECTION")

    if any(any(s in d for s in CMD_SIGNS) for d in norm_inputs):
        attacks.append("COMMAND_INJECTION")

    if any(any(s in d for s in PATH_SIGNS) for d in norm_inputs):
        attacks.append("PATH_TRAVERSAL")

    if any(any(s in d for s in XSS_SIGNS) for d in norm_inputs):
        attacks.append("XSS")

    return attacks

# =========================
# MAIN DETECTION FUNCTION
# =========================
def detect_attack_from_csv(row):
    url     = str(row.get("url", ""))
    body    = str(row.get("body", ""))
    headers = str(row.get("headers", ""))

    # 1. File upload check
    upload_result = check_file_upload_csv(headers, body)
    if upload_result == "FILE_UPLOAD_ABUSE":
        return "FILE_UPLOAD_ABUSE", 1

    # 2. Other attacks
    attacks = check_other_attacks_csv(url, body)

    if attacks:
        return "|".join(attacks), 1

    return "NORMAL", 0


In [41]:
df=pd.read_csv("final_attack_type.csv")

In [42]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,label
0,2025-01-01T12:10:00Z,offline,GET,http://test/fi?page=php://filter/convert.base6...,/fi,php://filter/convert.base64-encode/resource=in...,NORMAL,53.0,4.22,9.0,0
1,2025-01-01T12:11:00Z,offline,GET,http://test/fi?page=%2e%2e%2fetc%2fpasswd,/fi,%2e%2e%2fetc%2fpasswd,NORMAL,21.0,4.01,4.0,0
2,2025-01-01T12:12:00Z,offline,GET,http://test/fi?page=..%252f..%252fetc%252fpasswd,/fi,..%252f..%252fetc%252fpasswd,NORMAL,28.0,4.60,7.0,0
3,2025-01-01T12:13:00Z,offline,GET,http://test/fi?page=..%2f..%2f..%2fetc%2fpasswd,/fi,..%2f..%2f..%2fetc%2fpasswd,NORMAL,27.0,5.57,10.0,0
4,2025-01-01T12:14:00Z,offline,GET,http://test/fi?page=../../../../etc/passwd,/fi,../../../../etc/passwd,NORMAL,22.0,4.39,13.0,0


In [50]:
import pandas as pd
import re
import time
from urllib.parse import unquote_plus, urlparse, parse_qs

# =========================
# NORMALIZATION
# =========================
def normalize(text):
    if not isinstance(text, str):
        return ""
    for _ in range(3):
        text = unquote_plus(text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# =========================
# SIGNATURES
# =========================
SQLI_SIGNS = [" or 1=1", "union select", "sleep(", "benchmark(", "--"]
CMD_SIGNS  = [";", "&&", "||", "|", "`", "$("]
PATH_SIGNS = ["../", "..\\", "php://", "file://", "data://", "/etc/passwd"]
XSS_SIGNS  = ["<script", "</script>", "javascript:", "onerror=", "onload="]

ALLOWED_UPLOAD_EXT = [".png", ".jpg", ".jpeg", ".gif", ".pdf", ".txt","text"]
BLOCKED_UPLOAD_EXT = [".php", ".py", ".exe", ".jsp", ".sh",
                      "js",".bash",".c",".asm",".java",".class"]

# =========================
# LABEL MAP
# =========================
LABEL_MAP = {
    "SQL_INJECTION": 1,
    "COMMAND_INJECTION": 2,
    "PATH_TRAVERSAL": 3,
    "XSS": 4,
    "FILE_UPLOAD_ABUSE": 5,
    "DDOS": 6
}

# =========================
# DDOS CONFIG
# =========================
MAX_REQUESTS = 50
TIME_WINDOW = 60   # seconds

# =========================
# FILE UPLOAD CHECK (REFERENCE SAFE)
# =========================
def check_file_upload_csv(headers, body):
    if "multipart/form-data" not in headers.lower():
        return None

    matches = re.findall(r'filename="([^"]+)"', body, re.IGNORECASE)

    for fname in matches:
        fname = fname.lower()

        for ext in BLOCKED_UPLOAD_EXT:
            if fname.endswith(ext):
                return "FILE_UPLOAD_ABUSE"

        if fname.count(".") >= 2:
            return "FILE_UPLOAD_ABUSE"

        for ext in ALLOWED_UPLOAD_EXT:
            if fname.endswith(ext):
                return "NORMAL"

        return "FILE_UPLOAD_ABUSE"

    return "NORMAL"

# =========================
# OTHER ATTACKS
# =========================
def check_other_attacks_csv(url, body):
    attacks = []

    parsed = urlparse(url)
    inputs = []

    for v in parse_qs(parsed.query).values():
        inputs.extend(v)

    inputs.append(body)
    norm_inputs = [normalize(i) for i in inputs]

    if any(any(s in d for s in SQLI_SIGNS) for d in norm_inputs):
        attacks.append("SQL_INJECTION")

    if any(any(s in d for s in CMD_SIGNS) for d in norm_inputs):
        attacks.append("COMMAND_INJECTION")

    if any(any(s in d for s in PATH_SIGNS) for d in norm_inputs):
        attacks.append("PATH_TRAVERSAL")

    if any(any(s in d for s in XSS_SIGNS) for d in norm_inputs):
        attacks.append("XSS")

    return attacks

# =========================
# LOAD CSV
# =========================
df = pd.read_csv("final_attack_type.csv")

df["body"] = df.get("body", "")
df["headers"] = df.get("headers", "")

# timestamp MUST be parseable
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# =========================
# DDOS DETECTION (IP + TIME)
# =========================
df = df.sort_values("timestamp")
request_log = {}

ddos_flags = []

for _, row in df.iterrows():
    ip = row["client_ip"]
    ts = row["timestamp"]

    if pd.isna(ts):
        ddos_flags.append(False)
        continue

    now = ts.timestamp()
    request_log.setdefault(ip, [])

    request_log[ip] = [t for t in request_log[ip] if now - t <= TIME_WINDOW]
    request_log[ip].append(now)

    ddos_flags.append(len(request_log[ip]) > MAX_REQUESTS)

df["is_ddos"] = ddos_flags

# =========================
# FINAL LABEL FUNCTION
# =========================
def label_row(row):
    url     = str(row.get("url", ""))
    body    = str(row.get("body", ""))
    headers = str(row.get("headers", ""))

    attack_types = []
    labels = []

    # DDOS FIRST
    if row["is_ddos"]:
        attack_types.append("DDOS")
        labels.append(str(LABEL_MAP["DDOS"]))

    # FILE UPLOAD
    fu = check_file_upload_csv(headers, body)
    if fu == "FILE_UPLOAD_ABUSE":
        attack_types.append("FILE_UPLOAD_ABUSE")
        labels.append(str(LABEL_MAP["FILE_UPLOAD_ABUSE"]))

    # OTHER ATTACKS
    attacks = check_other_attacks_csv(url, body)
    for a in attacks:
        attack_types.append(a)
        labels.append(str(LABEL_MAP[a]))

    if not attack_types:
        return "NORMAL", 0

    return "|".join(attack_types), "|".join(labels)

# =========================
# APPLY LABELING
# =========================
df[["attack_type", "label"]] = df.apply(
    lambda r: pd.Series(label_row(r)),
    axis=1
)

df.to_csv("labeled_output.csv", index=False)


In [58]:
df=pd.read_csv("labeled_output.csv",low_memory=False)
df.columns

Index(['timestamp', 'client_ip', 'method', 'url', 'path', 'body',
       'attack_type', 'body_len', 'body_entropy', 'special_char_count',
       'label', 'headers', 'is_ddos'],
      dtype='object')

In [62]:
print(df['attack_type'].value_counts())

attack_type
NORMAL                              75848
SQL_INJECTION                        5765
COMMAND_INJECTION                    5115
SQL_INJECTION|COMMAND_INJECTION      4848
PATH_TRAVERSAL                       4009
XSS                                  3334
COMMAND_INJECTION|PATH_TRAVERSAL      692
COMMAND_INJECTION|XSS                   2
Name: count, dtype: int64


In [63]:
print(df['label'].value_counts())

label
0      75848
1       5765
2       5115
1|2     4848
3       4009
4       3334
2|3      692
2|4        2
Name: count, dtype: int64


In [64]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,label,headers,is_ddos
0,2025-01-01 10:00:00+00:00,offline,GET,http://test/cmd?q=cat /etc/passwd,/cmd,cat /etc/passwd,PATH_TRAVERSAL,15.0,3.67,3.0,3,NaN,False
1,2025-01-01 10:01:00+00:00,offline,GET,http://test/file?q=/etc/shadow,/file,/etc/shadow,NORMAL,11.0,4.56,2.0,0,NaN,False
2,2025-01-01 10:02:00+00:00,offline,POST,http://test/upload?q=filename=shell.php&conten...,/upload,filename=shell.php&content=<?php system($_GET[...,COMMAND_INJECTION,55.0,3.60,19.0,2,NaN,False
3,2025-01-01 10:03:00+00:00,offline,POST,http://test/upload?q=filename=test.jsp&content...,/upload,filename=test.jsp&content=<% Runtime.getRuntim...,COMMAND_INJECTION,85.0,4.91,22.0,2,NaN,False
4,2025-01-01 10:04:00+00:00,offline,GET,http://test/cmd?q=whoami,/cmd,whoami,NORMAL,6.0,3.96,0.0,0,NaN,False


In [65]:
#manual ddos

In [66]:
import csv
import random
import math
from datetime import datetime, timedelta
from collections import Counter

# ---------------- CONFIG ---------------- #

OUTPUT_FILE = "generated_attacks.csv"

DDOS_ROWS = 3000
FILE_UPLOAD_ROWS = 800

START_TIME = datetime(2025, 12, 31, 8, 0, 0)

TARGET_URL = "http://victim.local/login"
UPLOAD_URL = "http://target.local/upload.php"

DDOS_IPS = ["10.10.10.9", "10.10.10.10", "10.10.10.11"]

# --------------------------------------- #

LABEL_MAP = {
    "NORMAL": 0,
    "SQL_INJECTION": 1,
    "COMMAND_INJECTION": 2,
    "PATH_TRAVERSAL": 3,
    "XSS": 4,
    "FILE_UPLOAD_ABUSE": 5,
    "DDOS": 6
}

CSV_HEADER = [
    "timestamp",
    "client_ip",
    "method",
    "url",
    "path",
    "body",
    "attack_type",
    "body_len",
    "body_entropy",
    "special_char_count",
    "label",
    "headers",
    "is_ddos"
]

# ---------- FEATURE HELPERS ---------- #

def entropy(s: str) -> float:
    if not s:
        return 0.0
    counts = Counter(s)
    total = len(s)
    return round(-sum((c / total) * math.log2(c / total) for c in counts.values()), 3)

def special_char_count(s: str) -> int:
    return sum(1 for c in s if not c.isalnum())

# ---------- DDOS GENERATOR ---------- #

def generate_ddos(writer):
    current_time = START_TIME

    for _ in range(DDOS_ROWS):
        row = {
            "timestamp": current_time.isoformat() + "Z",
            "client_ip": random.choice(DDOS_IPS),
            "method": "GET",
            "url": TARGET_URL,
            "path": "/login",
            "body": "",
            "attack_type": "DDOS",
            "body_len": 0,
            "body_entropy": 0.0,
            "special_char_count": 0,
            "label": LABEL_MAP["DDOS"],
            "headers": "{}",
            "is_ddos": True
        }

        writer.writerow(row)

        # very small gap → burst traffic
        current_time += timedelta(milliseconds=random.randint(10, 80))

# ---------- FILE UPLOAD GENERATOR ---------- #

def generate_file_upload(writer):
    current_time = START_TIME + timedelta(minutes=10)

    filenames = [
        "shell.php.jpg",
        "cmd.jsp.png",
        "exploit.php",
        "payload.exe",
        "reverse.sh"
    ]

    for _ in range(FILE_UPLOAD_ROWS):
        filename = random.choice(filenames)
        payload = (
            "------WebKitFormBoundary\r\n"
            f"Content-Disposition: form-data; name=\"file\"; filename=\"{filename}\"\r\n"
            "Content-Type: application/octet-stream\r\n\r\n"
            "<?php system($_GET['cmd']); ?>\r\n"
            "------WebKitFormBoundary--"
        )

        row = {
            "timestamp": current_time.isoformat() + "Z",
            "client_ip": f"192.168.1.{random.randint(10, 200)}",
            "method": "POST",
            "url": UPLOAD_URL,
            "path": "/upload.php",
            "body": payload,
            "attack_type": "FILE_UPLOAD_ABUSE",
            "body_len": len(payload),
            "body_entropy": entropy(payload),
            "special_char_count": special_char_count(payload),
            "label": LABEL_MAP["FILE_UPLOAD_ABUSE"],
            "headers": "{\"Content-Type\":\"multipart/form-data\"}",
            "is_ddos": False
        }

        writer.writerow(row)
        current_time += timedelta(seconds=random.randint(5, 15))

# ---------- MAIN ---------- #

def main():
    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_HEADER)
        writer.writeheader()

        generate_ddos(writer)
        generate_file_upload(writer)

    print(f"Generated file: {OUTPUT_FILE}")
    print(f"DDOS rows: {DDOS_ROWS}")
    print(f"FILE_UPLOAD rows: {FILE_UPLOAD_ROWS}")

if __name__ == "__main__":
    main()


Generated file: generated_attacks.csv
DDOS rows: 3000
FILE_UPLOAD rows: 800


In [72]:
#combine this both
df1=pd.read_csv("generated_attacks.csv")
df2=pd.read_csv("labeled_output.csv")
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined dataframe to a new CSV file
combined_df.to_csv('combined_file.csv', index=False)

/tmp/ipykernel_9169/1812922080.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv("labeled_output.csv")


In [70]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,label,headers,is_ddos
0,2025-01-01 10:00:00+00:00,offline,GET,http://test/cmd?q=cat /etc/passwd,/cmd,cat /etc/passwd,PATH_TRAVERSAL,15.0,3.67,3.0,3,NaN,False
1,2025-01-01 10:01:00+00:00,offline,GET,http://test/file?q=/etc/shadow,/file,/etc/shadow,NORMAL,11.0,4.56,2.0,0,NaN,False
2,2025-01-01 10:02:00+00:00,offline,POST,http://test/upload?q=filename=shell.php&conten...,/upload,filename=shell.php&content=<?php system($_GET[...,COMMAND_INJECTION,55.0,3.60,19.0,2,NaN,False
3,2025-01-01 10:03:00+00:00,offline,POST,http://test/upload?q=filename=test.jsp&content...,/upload,filename=test.jsp&content=<% Runtime.getRuntim...,COMMAND_INJECTION,85.0,4.91,22.0,2,NaN,False
4,2025-01-01 10:04:00+00:00,offline,GET,http://test/cmd?q=whoami,/cmd,whoami,NORMAL,6.0,3.96,0.0,0,NaN,False


In [76]:
new=pd.read_csv("combined_file.csv")
new.head()

/tmp/ipykernel_9169/3042380344.py:1: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  new=pd.read_csv("combined_file.csv")


,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,label,headers,is_ddos
0,2025-12-31T08:00:00Z,10.10.10.10,GET,http://victim.local/login,/login,NaN,DDOS,0.0,0.0,0.0,6,{},True
1,2025-12-31T08:00:00.014000Z,10.10.10.11,GET,http://victim.local/login,/login,NaN,DDOS,0.0,0.0,0.0,6,{},True
2,2025-12-31T08:00:00.071000Z,10.10.10.10,GET,http://victim.local/login,/login,NaN,DDOS,0.0,0.0,0.0,6,{},True
3,2025-12-31T08:00:00.137000Z,10.10.10.9,GET,http://victim.local/login,/login,NaN,DDOS,0.0,0.0,0.0,6,{},True
4,2025-12-31T08:00:00.169000Z,10.10.10.10,GET,http://victim.local/login,/login,NaN,DDOS,0.0,0.0,0.0,6,{},True


In [77]:
import pandas as pd
from datetime import timedelta

# ---------------- CONFIG ---------------- #

INPUT_CSV = "combined_file.csv"          # your original file
OUTPUT_CSV = "output_fixed.csv"  # new cleaned file

# Base timestamp for filling missing values
FILL_START = pd.Timestamp("2025-12-31T08:00:00.071000Z")

# Increment between generated timestamps
FILL_INCREMENT_MS = 500  # milliseconds

# -------------------------------------- #

def normalize_timestamp(ts):
    """
    Convert timestamp to ISO 8601 Z format:
    YYYY-MM-DDTHH:MM:SS.mmmmmmZ
    """
    if pd.isna(ts) or str(ts).strip() == "":
        return None

    try:
        ts = pd.to_datetime(ts, utc=True)
        return ts.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    except Exception:
        return None


def main():
    df = pd.read_csv(INPUT_CSV)

    # Normalize existing timestamps
    df["timestamp"] = df["timestamp"].apply(normalize_timestamp)

    # Identify missing timestamps
    missing_mask = df["timestamp"].isna()

    # Start filling missing timestamps
    current_ts = FILL_START

    for idx in df[missing_mask].index:
        df.at[idx, "timestamp"] = current_ts.strftime(
            "%Y-%m-%dT%H:%M:%S.%fZ"
        )
        current_ts += timedelta(milliseconds=FILL_INCREMENT_MS)

    # Safety check
    if df["timestamp"].isna().any():
        raise ValueError("Some timestamps could not be filled!")

    # Write output
    df.to_csv(OUTPUT_CSV, index=False)

    print("Timestamp normalization complete.")
    print(f"Input file : {INPUT_CSV}")
    print(f"Output file: {OUTPUT_CSV}")
    print(f"Filled {missing_mask.sum()} missing timestamps.")


if __name__ == "__main__":
    main()


/tmp/ipykernel_9169/1181997752.py:33: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)


Timestamp normalization complete.
Input file : combined_file.csv
Output file: output_fixed.csv
Filled 66613 missing timestamps.


In [79]:
new=pd.read_csv("output_fixed.csv",low_memory=False)

In [83]:
print(new["body"].isnull().sum())

3000


In [95]:
print(new["headers"])

0          {}
1          {}
2          {}
3          {}
4          {}
         ... 
103408    NaN
103409    NaN
103410    NaN
103411    NaN
103412    NaN
Name: headers, Length: 103413, dtype: object


In [96]:
new=new.drop(columns=['headers'])

In [98]:
new.to_csv("without_NAN.csv")#################################

In [93]:
print(new.dtypes)

timestamp              object
client_ip              object
method                 object
url                    object
path                   object
body                   object
attack_type            object
body_len              float64
body_entropy          float64
special_char_count    float64
label                  object
headers                object
is_ddos                  bool
dtype: object


In [99]:
print(new['attack_type'].value_counts())

attack_type
NORMAL                              75848
SQL_INJECTION                        5765
COMMAND_INJECTION                    5115
SQL_INJECTION|COMMAND_INJECTION      4848
PATH_TRAVERSAL                       4009
XSS                                  3334
DDOS                                 3000
FILE_UPLOAD_ABUSE                     800
COMMAND_INJECTION|PATH_TRAVERSAL      692
COMMAND_INJECTION|XSS                   2
Name: count, dtype: int64


In [103]:
print(new['label'].value_counts())
new['label'] = new['label'].astype(str)


label
0      75848
1       5765
2       5115
1|2     4848
3       4009
4       3334
6       3000
5        800
2|3      692
2|4        2
Name: count, dtype: int64


In [105]:
import numpy as np

NUM_CLASSES = 7

def encode_label(label):
    vec = np.zeros(NUM_CLASSES, dtype=int)
    for l in label.split('|'):
        vec[int(l)] = 1
    return vec

new['labels'] = new['label'].apply(encode_label)


In [106]:
y = np.stack(new['labels'].values)


In [151]:
new=new.drop(columns=["label","encoded_label"],inplace=True)
new.head()

KeyError: "['label', 'encoded_label'] not found in axis"

In [121]:
new.to_csv("without_NAN_label.csv")

In [120]:
new.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,is_ddos,labels
0,2025-12-31T08:00:00.000000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,"[0, 0, 0, 0, 0, 0, 1]"
1,2025-12-31T08:00:00.014000Z,10.10.10.11,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,"[0, 0, 0, 0, 0, 0, 1]"
2,2025-12-31T08:00:00.071000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,"[0, 0, 0, 0, 0, 0, 1]"
3,2025-12-31T08:00:00.137000Z,10.10.10.9,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,"[0, 0, 0, 0, 0, 0, 1]"
4,2025-12-31T08:00:00.169000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,"[0, 0, 0, 0, 0, 0, 1]"


In [2]:
import pandas as pd
df=pd.read_csv("without_NAN_label.csv")
df.columns

Index(['timestamp', 'client_ip', 'method', 'url', 'path', 'body',
       'attack_type', 'body_len', 'body_entropy', 'special_char_count',
       'is_ddos', 'labels'],
      dtype='object')

In [3]:

df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,is_ddos,labels
0,2025-12-31T08:00:00.000000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
1,2025-12-31T08:00:00.014000Z,10.10.10.11,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
2,2025-12-31T08:00:00.071000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
3,2025-12-31T08:00:00.137000Z,10.10.10.9,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
4,2025-12-31T08:00:00.169000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]


In [170]:
df.dtypes

timestamp              object
client_ip              object
method                 object
url                    object
path                   object
body                   object
attack_type            object
body_len              float64
body_entropy          float64
special_char_count    float64
is_ddos                  bool
labels                 object
dtype: object

In [166]:
X = df.drop(columns=['attack_type', 'labels'])
y = np.stack(df['labels'].values)


In [167]:
print(df['labels'].value_counts())

labels
[1 0 0 0 0 0 0]    75848
[0 1 0 0 0 0 0]     5765
[0 0 1 0 0 0 0]     5115
[0 1 1 0 0 0 0]     4848
[0 0 0 1 0 0 0]     4009
[0 0 0 0 1 0 0]     3334
[0 0 0 0 0 0 1]     3000
[0 0 0 0 0 1 0]      800
[0 0 1 1 0 0 0]      692
[0 0 1 0 1 0 0]        2
Name: count, dtype: int64


In [168]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,is_ddos,labels
0,2025-12-31T08:00:00.000000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
1,2025-12-31T08:00:00.014000Z,10.10.10.11,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
2,2025-12-31T08:00:00.071000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
3,2025-12-31T08:00:00.137000Z,10.10.10.9,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]
4,2025-12-31T08:00:00.169000Z,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,True,[0 0 0 0 0 0 1]


In [7]:
#dataset clening
df.dtypes

timestamp             datetime64[ns]
client_ip                     object
method                        object
url                           object
path                          object
body                          object
attack_type                   object
body_len                     float64
body_entropy                 float64
special_char_count           float64
is_ddos                         bool
labels                        object
year                           int32
month                          int32
day                            int32
hour                           int32
minute                         int32
second                         int32
weekday                        int32
day_of_week                   object
dtype: object

In [6]:
# Convert the 'timestamp' column to datetime (handling the 'Z' for UTC)
#df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')


df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df['weekday'] = df['timestamp'].dt.weekday  # 0 = Monday, 6 = Sunday
df['day_of_week'] = df['timestamp'].dt.day_name()

In [19]:
import pandas as pd
import re


# Function to clean and extract valid IP addresses
def clean_ip(ip):
    # If the value is 'offline', replace it with a default IP address
    if ip == 'offline' or pd.isnull(ip):
        return '0.0.0.0'  # Replace 'offline' with '0.0.0.0' or any default IP you prefer
    # If it's in the tuple format, extract the IP (the first element of the tuple)
    elif isinstance(ip, str) and ip.startswith('('):
        # Extract IP using regex (this will match the IP before the comma)
        match = re.match(r"\('([^']+)'", ip)
        if match:
            return match.group(1)
    # If it's a valid IP address (no tuple, no 'offline')
    elif isinstance(ip, str) and re.match(r"^\d{1,3}(\.\d{1,3}){3}$", ip):
        return ip
    # Return a default IP for anything else (invalid data)
    return '0.0.0.0'

# Apply the function to the 'client_ip' column
df['client_ip'] = df['client_ip'].apply(clean_ip)

# Convert the 'client_ip' column to string type (object type in pandas)
df['client_ip'] = df['client_ip'].astype(str)

# Save the cleaned dataset
df.to_csv("cleaned_ip_file.csv", index=False)

# Display the cleaned dataframe
print(df.head())  # Show the first few rows to confirm the changes


                timestamp    client_ip method                        url  \
0 2025-12-31 08:00:00.000  10.10.10.10    GET  http://victim.local/login   
1 2025-12-31 08:00:00.014  10.10.10.11    GET  http://victim.local/login   
2 2025-12-31 08:00:00.071  10.10.10.10    GET  http://victim.local/login   
3 2025-12-31 08:00:00.137   10.10.10.9    GET  http://victim.local/login   
4 2025-12-31 08:00:00.169  10.10.10.10    GET  http://victim.local/login   

     path     body attack_type  body_len  body_entropy  special_char_count  \
0  /login  UNKNOWN        DDOS       0.0           0.0                 0.0   
1  /login  UNKNOWN        DDOS       0.0           0.0                 0.0   
2  /login  UNKNOWN        DDOS       0.0           0.0                 0.0   
3  /login  UNKNOWN        DDOS       0.0           0.0                 0.0   
4  /login  UNKNOWN        DDOS       0.0           0.0                 0.0   

   is_ddos           labels  year  month  day  hour  minute  second  weekd

In [16]:
#encode thee cleint_ip

In [26]:
#%pip install 
#from sklearn.preprocessing import StandardScaler   
from sklearn.preprocessing import LabelEncoder


In [27]:
label_encoder = LabelEncoder()

# Fit and transform the 'client_ip' column to numeric labels
df['client_ip_encoded'] = label_encoder.fit_transform(df['client_ip'])


In [28]:
df.to_csv("cleaned_ip_file.csv",index=False)

In [1]:
import pandas as pd
df=pd.read_csv("cleaned_ip_file.csv")
df.head()
print(df['client_ip_encoded'].value_counts())

client_ip_encoded
4      89256
0       9500
2       1012
3       1011
1        977
       ...  
39         1
103        1
149        1
62         1
98         1
Name: count, Length: 197, dtype: int64


In [3]:
df['attack_type'].value_counts()
df['labels'].value_counts()

labels
[1 0 0 0 0 0 0]    75848
[0 1 0 0 0 0 0]     5765
[0 0 1 0 0 0 0]     5115
[0 1 1 0 0 0 0]     4848
[0 0 0 1 0 0 0]     4009
[0 0 0 0 1 0 0]     3334
[0 0 0 0 0 0 1]     3000
[0 0 0 0 0 1 0]      800
[0 0 1 1 0 0 0]      692
[0 0 1 0 1 0 0]        2
Name: count, dtype: int64

In [3]:
df.head()

,timestamp,client_ip,method,url,path,body,attack_type,body_len,body_entropy,special_char_count,...,labels,year,month,day,hour,minute,second,weekday,day_of_week,client_ip_encoded
0,2025-12-31 08:00:00.000,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,...,[0 0 0 0 0 0 1],2025,12,31,8,0,0,2,Wednesday,1
1,2025-12-31 08:00:00.014,10.10.10.11,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,...,[0 0 0 0 0 0 1],2025,12,31,8,0,0,2,Wednesday,2
2,2025-12-31 08:00:00.071,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,...,[0 0 0 0 0 0 1],2025,12,31,8,0,0,2,Wednesday,1
3,2025-12-31 08:00:00.137,10.10.10.9,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,...,[0 0 0 0 0 0 1],2025,12,31,8,0,0,2,Wednesday,3
4,2025-12-31 08:00:00.169,10.10.10.10,GET,http://victim.local/login,/login,UNKNOWN,DDOS,0.0,0.0,0.0,...,[0 0 0 0 0 0 1],2025,12,31,8,0,0,2,Wednesday,1


In [17]:
#method data
#one-hot encode
# One-Hot Encoding for 'method' column (GET and POST)
df= pd.get_dummies(df, columns=['method'], prefix=['method'])


In [18]:
print(df['method_POST'].value_counts())

method_POST
True     53730
False    49683
Name: count, dtype: int64


In [19]:
df.dtypes

timestamp              object
client_ip              object
url                    object
path                   object
body                   object
attack_type            object
body_len              float64
body_entropy          float64
special_char_count    float64
is_ddos                  bool
labels                 object
year                    int64
month                   int64
day                     int64
hour                    int64
minute                  int64
second                  int64
weekday                 int64
day_of_week            object
client_ip_encoded       int64
method_GET               bool
method_POST              bool
dtype: object

In [8]:
#url TF-IDF text vectoreization

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [11]:
# Step 2: Clean the `url` column (optional)
# Removing any non-alphanumeric characters, keeping only letters and numbers in the URL
df['url_cleaned'] = df['url'].str.replace(r'\W', ' ', regex=True)


In [ ]:
# Step 2: Clean the `url` column (optional but helpful)
# Remove any non-alphanumeric characters, keeping only letters and numbers
df['url_cleaned'] = df['url'].str.replace(r'\W', ' ', regex=True)

# Step 3: Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

# Step 4: Fit and transform the `url_cleaned` column to get the TF-IDF matrix
X = vectorizer.fit_transform(df['url_cleaned'])

# Step 5: Convert the result into a DataFrame with feature names as columns
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Step 6: Concatenate the original DataFrame with the new vectorized data
df_combined = pd.concat([df, X_df], axis=1)

# Step 7: Save the updated DataFrame back to the CSV (without overwriting)
df_combined.to_csv("url_cleaned_file.csv", index=False)  # Replace with your desired output file name

In [15]:
df.head()

,0a,0aecho,0aelse,0afi,0aif,0astr,0astr1,0athen,0d,10,...,xhenqf,xml,xmlrpc,xmlrpc_server,xss,xss_r,xxx,yytwua,zh,zptaks
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
